In [1]:
from resnet18 import *

In [65]:
ourbestmodel = torch.load('./models/best_resnet_cropped_full_linda.pt')
# anothermodel = torch.load('./models/best_resnet18_nonprocessedimg.pt')

In [61]:
def evalModel(model, evalPath, raw=False):
    device = torch.device('cuda:0')
    criterion = nn.CrossEntropyLoss()
    model.eval()   # Set model to evaluate mode
    if raw:
        image_transforms = {
            'test': transforms.Compose([
            transforms.Resize(256),
            transforms.CenterCrop(224),
            transforms.ToTensor(),
            transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
            ])
        }
    else: #segmented
        image_transforms = {
            'test': transforms.Compose([
                transforms.Resize(256),
                transforms.CenterCrop(224),
                transforms.ToTensor(),
                transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
        ])
        }
    data = {
        'test': datasets.ImageFolder(root=evalPath, transform=image_transforms['test'])
    }
    dataloaders = {
           'test': torch.utils.data.DataLoader(data['test'], batch_size=32, shuffle=True)
    }

    running_loss = 0
    running_corrects = 0
    running_labels = []
    running_preds = []

    for inputs, labels in dataloaders['test']:
        inputs = inputs.to(device)
        labels = labels.to(device)    
        with torch.set_grad_enabled(False):
            outputs = model(inputs)
            _, preds = torch.max(outputs, 1)
            loss = criterion(outputs, labels)
            preds[preds==0] = 2
            preds[preds==4] = 2
        # statistics
        running_loss += loss.item() * inputs.size(0)
        running_corrects += torch.sum(preds == labels.data)
        running_labels.extend(list(labels.data.cpu()))
        running_preds.extend(list(preds.cpu()))
    print(data['test'].classes)
    loss = running_loss / len(data['test'])
    acc = running_corrects.double() / len(data['test'])
    cm = makeConfusionMatrixTest(np.array(running_labels, dtype=np.int32), np.array(running_preds, dtype=np.int32))
    print(cm)
    sens, spec = getSensitivityAndSpecificity(cm)
    print('{} Loss: {:.4f} Acc: {:.4f}'.format('Test', loss, acc))
    print('Sensitivity: {:.4f} Specificity {:.4f}'.format(sens, spec))
    return

In [56]:
def makeConfusionMatrixTest(labels, preds, normalize=False):

    cm = confusion_matrix(labels, preds, labels=[0,1,2,3,4])#['1pn', '2pn', '3pn', 'gv', 'nf']
    if normalize:
        cm = cm.astype('float') / cm.sum(axis=1)[:, np.newaxis]
    return cm


In [66]:
evalModel(ourbestmodel, './dataset/val/')

AttributeError: 'Conv2d' object has no attribute 'padding_mode'

In [54]:
evalModel(anothermodel, './mock_test/')

<class 'torch.Tensor'>
<class 'torch.Tensor'>
<class 'torch.Tensor'>
<class 'torch.Tensor'>
<class 'torch.Tensor'>
<class 'torch.Tensor'>
<class 'torch.Tensor'>
<class 'torch.Tensor'>
['1pn', '2pn', '3pn', 'gv', 'nf']
[[  0   0   0   0   0]
 [  0 199   5   0   0]
 [  0   4  15   1   0]
 [  0   1   0   3   0]
 [  0   0   0   0   0]]
Test Loss: 0.1546 Acc: 0.9518
Sensitivity: 0.9755 Specificity 0.7917


In [62]:
evalModel(anothermodel, './test_set/')

['1pn', '2pn', '3pn', 'gv', 'nf']
[[ 0  0  0  0  0]
 [ 0 15 56  2  0]
 [ 0  3 18  0  0]
 [ 0  1  5  0  0]
 [ 0  0  0  0  0]]
Test Loss: 1.2347 Acc: 0.3300
Sensitivity: 0.2055 Specificity 0.8519


In [68]:
torch.__version__

'1.1.0'